In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [3]:
import tensorflow
print(tensorflow.__version__)

1.15.2


# **Cargamos Inception**

In [9]:
from keras.applications import inception_v3
from keras import backend as K

# Debido a que nuestro modelo no será entrenado,
# procederemos a deshabilitar todas las operaciones relacionadas con el entrenamiento
K.set_learning_phase(0)

# Cargamos InceptionV3.
# La carga se realizará considerando los pesos pre-entrenados en ImageNet
model = inception_v3.InceptionV3(weights='imagenet',
                                 include_top=False)

In [10]:
model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv2d_283 (Conv2D)             (None, None, None, 3 864         input_4[0][0]                    
__________________________________________________________________________________________________
batch_normalization_283 (BatchN (None, None, None, 3 96          conv2d_283[0][0]                 
__________________________________________________________________________________________________
activation_283 (Activation)     (None, None, None, 3 0           batch_normalization_283[0][0]    
_______________________________________________________________________________________

# **Layer names and contributions**

In [11]:
# Diccionario que asocia los nombres de las capas con un coeficiente correspondiente,
# el cual determina cuanto contribuye la activación de cada capa a la pérdida
# que buscamos maximizar.
# El nombre de las capas se especifica tal y como aparecen en InceptionV3.
# Estos nombres pueden consultarse usando 'model.summary()' 
#Carpeta Bisquets
'''
layer_contributions = {
    'mixed7': 0.2,
    'mixed8': 3.,
    'mixed9': 2.,
    'mixed10': 1.5,
}

#Carpeta Bisquets2
layer_contributions = {
    'mixed7': 0.2,
    'mixed8': 1.8,
    'mixed9': 1.,
    'mixed10': 0.5,
}
#Carpeta Bisquets3
layer_contributions = {
    'mixed7': 0.2,
    'mixed8': 0.9,
    'mixed9': 1.,
    'mixed10': 0.5,
}'''
#Carpeta Bisquets4
layer_contributions = {
    'mixed6': 0.2,
    'mixed7': 0.9,
    'mixed8': 1.,
    'mixed9': 0.5,
}

# **Calculamos la pérdida**

In [ ]:
# Obtenemos la activación de cada capa
layer_dict = dict([(layer.name, layer) for layer in model.layers])

# Definimos la pérdida.
loss = K.variable(0.)
for layer_name in layer_contributions:

    coeff = layer_contributions[layer_name]
    activation = layer_dict[layer_name].output

    # En la pérdida sólo se incluyen los píxeles que no pertenezcan a los bordes, 
    # pues los píxeles presentes en los bordes pueden agregar ruido.
    scaling = K.prod(K.cast(K.shape(activation), 'float32'))
    loss = loss + coeff * K.sum(K.square(activation[:, 2: -2, 2: -2, :])) / scaling #Agregamos las contribuciones de cada capa a la pérdida

In [ ]:
#Disable eager execution
import tensorflow as tf
tf.compat.v1.disable_eager_execution()

# **Función para aplicar ascenso de gradiente**

In [ ]:
# Esta variable almacena la imagen generada, es decir, el sueño.
dream = model.input

# Calculamos los gradiente del sueño con respecto a la pérdida.
grads = K.gradients(loss, dream)[0]

# Normalizamos los gradientes.
grads /= K.maximum(K.mean(K.abs(grads)), 1e-7)

# Configuramos la función para obtener el valor
# de la pérdida y de los gradientes dada una imagen de entrada.
outputs = [loss, grads]
fetch_loss_and_grads = K.function([dream], outputs)

def eval_loss_and_grads(x):
    outs = fetch_loss_and_grads([x])
    loss_value = outs[0]
    grad_values = outs[1]
    return loss_value, grad_values

def gradient_ascent(x, iterations, step, max_loss=None):
    for i in range(iterations):
        loss_value, grad_values = eval_loss_and_grads(x)
        if max_loss is not None and loss_value > max_loss:
            break
        print('...Valor de la pérdida en la iteración ', i, ':', loss_value)
        x += step * grad_values
    return x

# **Funciones útiles**

In [ ]:
import scipy
import imageio
from keras.preprocessing import image

# Redimensionamiento de imágenes
def resize_img(img, size):
    img = np.copy(img)
    factors = (1,
               float(size[0]) / img.shape[1],
               float(size[1]) / img.shape[2],
               1)
    return scipy.ndimage.zoom(img, factors, order=1)

#Almacenamiento de imágenes
def save_img(img, fname):
    pil_img = deprocess_image(np.copy(img))
    imageio.imwrite(fname, pil_img)

# Esta función nos sirve para abrir, redimensionar y
# convertir una imagen en un tensor.
def preprocess_image(image_path):
    img = image.load_img(image_path)
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = inception_v3.preprocess_input(img)
    return img

# Esta función nos permite convertir un tensor en una imagen
def deprocess_image(x):
    if K.image_data_format() == 'channels_first':
        x = x.reshape((3, x.shape[2], x.shape[3]))
        x = x.transpose((1, 2, 0))
    else:
        x = x.reshape((x.shape[1], x.shape[2], 3))
    x /= 2.
    x += 0.5
    x *= 255.
    x = np.clip(x, 0, 255).astype('uint8')
    return x

# **Algoritmo DeepDream**

**Parte 1**

In [ ]:
import numpy as np

# Además de las capas, estos hiperparámetros también te permitirán obtener distintos efectos.
step = 0.08  # Tasa de aprendizaje del ascenso de gradiente
num_octave = 3  # Número de escalas en las que se ejecutará el ascenso de gradiente
octave_scale = 1.4  # Incremento entre escalas
iterations = 20  # Número de aplicaciones del ascenso de gradiente por cada escala

# Si el valor de la pérdida es mayor a 10,
# detendremos el proceso de ascenso de gradiente, para evitar efectos indeseados.
max_loss = 20.

# Ruta a la imagen que será modificada
base_image_path = '/content/castle.jpg'
#base_image_path = 'oceano.jpeg'

# Cargamos la imagen a un numpy array
img = preprocess_image(base_image_path)

**Parte 2**

In [ ]:
# Preparamos una lista de tuplas, donde cada tupla
# almacenará las diferentes escalas en las cuales tiene que aplicarse el ascenso de gradiente
original_shape = img.shape[1:3]
successive_shapes = [original_shape]
for i in range(1, num_octave):
    shape = tuple([int(dim / (octave_scale ** i)) for dim in original_shape])
    successive_shapes.append(shape)

# Invertimos la lista de dimensiones, de tal manera que tengan un orden creciente
successive_shapes = successive_shapes[::-1]

In [ ]:
successive_shapes

[(221, 351), (310, 491), (435, 688)]

**Parte 3**

In [ ]:
# Redimensionamos el numpy array de la imagen a la escala más pequeña
original_img = np.copy(img)
shrunk_original_img = resize_img(img, successive_shapes[0])

In [ ]:
shrunk_original_img.shape

(1, 146, 261, 3)

**Parte 4**

In [ ]:
import os
dest = '/content/drive/My Drive/DeepDream/Castle1'
os.mkdir(dest)

In [ ]:
for shape in successive_shapes:
    print('Processing image shape', shape)
    img = resize_img(img, shape)
    img = gradient_ascent(img,
                          iterations=iterations,
                          step=step,
                          max_loss=max_loss)
    upscaled_shrunk_original_img = resize_img(shrunk_original_img, shape) #pixellated
    print('upscaled_shrunk_original_img:',upscaled_shrunk_original_img.shape)
    same_size_original = resize_img(original_img, shape)
    print('same_size_original: ',same_size_original.shape)
    lost_detail = same_size_original - upscaled_shrunk_original_img

    img += lost_detail
    shrunk_original_img = resize_img(original_img, shape)
    print('shrunk_original_img: ', shrunk_original_img.shape)
    save_img(img, dest +'/dream_at_scale_' + str(shape) + '.png')
save_img(img, dest + '/final_dream.png')

Processing image shape (458, 687)
...Valor de la pérdida en la iteración  0 : 0.15174334
...Valor de la pérdida en la iteración  1 : 0.38987622
...Valor de la pérdida en la iteración  2 : 0.6765182
...Valor de la pérdida en la iteración  3 : 0.87820566
...Valor de la pérdida en la iteración  4 : 1.1563272
...Valor de la pérdida en la iteración  5 : 1.5878203
...Valor de la pérdida en la iteración  6 : 2.2119048
...Valor de la pérdida en la iteración  7 : 3.7936795
...Valor de la pérdida en la iteración  8 : 6.7426
...Valor de la pérdida en la iteración  9 : 10.80856
...Valor de la pérdida en la iteración  10 : 15.299954
upscaled_shrunk_original_img: (1, 458, 687, 3)
same_size_original:  (1, 458, 687, 3)
shrunk_original_img:  (1, 458, 687, 3)
Processing image shape (642, 962)
...Valor de la pérdida en la iteración  0 : 1.4682088
...Valor de la pérdida en la iteración  1 : 5.767294
...Valor de la pérdida en la iteración  2 : 14.442814
upscaled_shrunk_original_img: (1, 642, 962, 3)
same_s

In [ ]:
upscaled_shrunk_original_img = resize_img(shrunk_original_img, shape)
upscaled_shrunk_original_img.shape

(1, 288, 512, 3)